In [121]:
from abc import ABCMeta, abstractmethod
import numpy as np
import matplotlib.pyplot as plt
from typing import *
from typing import Tuple
import itertools as it
from sympy.solvers import *
from sympy import Symbol, Poly, Eq

def distance(point1: np.ndarray, point2: np.ndarray) -> float:
    sq = 0
    for x1, x2 in zip(point1, point2):
        sq += (x1 - x2)**2
    return np.sqrt(sq)

def angle(point1: np.ndarray, point2: np.ndarray, point3: np.ndarray) -> float:
        ba = point2 - point1
        bc = point2 - point3
        out =  np.degrees(np.arccos(np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))))
        return out

class Cluster(metaclass=ABCMeta):
    def __init__(self, center_coordinates: np.ndarray, shape: np.ndarray, step: float):
        self.center = center_coordinates
        if any([size == 0 for size in shape]):
            raise ValueError('Shape of a cluster can not contain 0')
        self.shape = shape
        self.step = step
        self.volume = None

    def __str__(self):
        return '{}-dimensional cluster\n ' \
               'with center in {}\n ' \
               'and with shape {}'.format(self.center.shape[0], self.center, self.shape*2)
    def __mul__(self, num: int):
        self.shape = self.shape*num
        return self.shape
    def __add__(self, coordinate: np.ndarray):
        self.center = np.add(self.center, coordinate)
        return self.center
    def __len__(self):
        return len(self.center)
    def __lt__(self, other):
        return self.volume < other.volume
    def __gt__(self, other):
        return self.volume > other.volume
    def __le__(self, other):
        return self.volume <= other.volume
    def __ge__(self, other):
        return self.volume >= other.volume
    def __eq__(self, other):
        return self.volume == other.volume
    def __contains__(self, item: np.ndarray):
        return self.includes(item)

    @property
    def center(self):
        return self
    @center.setter
    def center(self, center_coordinates: np.ndarray):
        self._center = center_coordinates
    @center.getter
    def center(self):
        return self._center

    @property
    def shape(self):
        return self
    @shape.setter
    def shape(self, shape: np.ndarray):
        self._shape = shape
    @shape.getter
    def shape(self):
        return self._shape

    @property
    def step(self):
        return self
    @step.setter
    def step(self, value):
      self._step = value
    @step.getter
    def step(self):
        return self._step

    @property
    def volume(self):
        return self
    @volume.setter
    def volume(self, value):
      self._volume = value
    @volume.getter
    def volume(self):
        return self._volume

    @abstractmethod
    def includes(self, point: np.ndarray) -> bool:
        pass
    @abstractmethod
    def compute_volume(self):
        pass

    def radius_to(self, point: np.ndarray, iters: int = 100) -> Tuple[np.ndarray, float, float]:
        def find_edge(cluster: Cluster, step_price: np.ndarray, r: float, iters: int) -> np.ndarray:
            diff = r
            for i in range(iters):
                diff = diff/2
                now = cluster.center + step_price*r
                if now in cluster:
                    while now in cluster:
                        r += diff
                        now = cluster.center + step_price*r
                        if now in self and distance(self.center, now) > 5:
                            print("in cluster: ", now, '\n', 'distance: ', distance(self.center, now), '\n*********')
                        if round(diff, 10) == 0:
                            return now
                else:
                    while now not in cluster:
                        r -= diff
                        now = cluster.center + step_price*r
                        if round(diff, 10) == 0:
                            return now
            return cluster.center + step_price*r

        step_price = point - self.center
        max_r = np.sqrt(np.sum(self.shape**2))
        coordinates = find_edge(self, step_price, max_r, iters)
        return \
            coordinates,\
            distance(self.center, coordinates),\
            distance(coordinates, point)


class HyperEllipseCluster(Cluster):
    def __init__(self, center_coordinates: np.ndarray, shape: np.ndarray, step: float = 0.001):
        super().__init__(center_coordinates, shape, step)
        self.volume = self.volume_with_int()
    def __str__(self):
        return '{}-dimensional elliptic cluster\n ' \
               'with center in {}\n ' \
               'and with shape {}'.format(self.center.shape[0], self.center, self.shape*2)
    def __iter__(self):
        self.n = 0
        self.c = self.center
        return self
    def __next__(self):
        # recursive solution impossible (max recursion depth is reached)
        while self.n < (1/self.step)**self.center.shape[0]:
            self.n += 1
            self.c = np.array([
                c + (s * self.step) * (
                        self.n % (1/self.step)**i // (1/self.step)**(i-1)
                ) for c, s, i in zip(
                    self.center, self.shape, range(self.center.shape[0], 0, -1)
                )
            ])
            if self.c in self:
                return self.c
        else:
            raise StopIteration

    def includes(self, point: np.ndarray) -> bool:
        eq=0
        for x, r in zip(point, self.shape):
            eq+=(x**2/r**2)
        if eq > 1:
            return False
        else:
            return True


    def compute_volume(self):
        dv = np.prod(self.shape*self.step)
        v = 0
        for i in self:
            v += dv
        return v*(2**self.center.shape[0])

    def volume_with_int(self):
        parallelepiped = np.prod(self.shape)
        dimensional_transition = 2*np.pi
        if self.shape.shape[0]%2 != 0:
            spherical_ratio = 2
            for i in range(1, self.shape.shape[0], 2):
                spherical_ratio *= dimensional_transition/(i + 2)
        else:
            spherical_ratio = 1
            for i in range(0, self.shape.shape[0], 2):
                spherical_ratio *= dimensional_transition/(i + 2)
        return parallelepiped*spherical_ratio

class ClustersStatistic(object):
    def __init__(self, clusters: List[HyperEllipseCluster]):
        for i, j in list(it.combinations(range(len(clusters)), 2)):
            if clusters[i].center.shape != clusters[j].center.shape:
                raise ValueError('Clusters must have equal dimensions')
            if all(clusters[i].center == clusters[j].center):
                raise ValueError('Clusters {} and {} have equal center coordinates. All the next computations have no any sense'.format(i, j))
        self.clusters = clusters
        self.distances = {
            'distance_between_centers_of_{}_and_{}'.format(i, j):
            distance(clusters[i].center, clusters[j].center)
            for i, j in list(it.combinations(range(len(clusters)), 2))
        }
        self.edges = {
            'edges_for_{}_and_{}_common_axis'.format(i, j):
            self.compute_edges(clusters[i], clusters[j])
            for i, j in list(it.combinations(range(len(clusters)), 2))

        }
    @staticmethod
    def compute_edges(cluster1: Cluster, cluster2: Cluster, iters: int = 100):
        return (
            cluster1.radius_to(cluster2.center, iters),
            cluster2.radius_to(cluster1.center, iters)
        )



cls1 = HyperEllipseCluster(np.array([2, 2]), np.array([5, 5]))
cls2 = HyperEllipseCluster(np.array([-2, -2]), np.array([1, 1]))

def make_symbols(cluster):
    symbols = list()
    for i in range(len(cluster)):
        symbols.append(Symbol('x{}'.format(i)))
    return symbols

def make_eq(symbols, cluster):
    return sum([(symbol - x0)**2/r**2 for symbol, x0, r in zip(symbols, cluster.center, cluster.shape)])

def solve_eq(eq1, eq2):
    return solve(Eq(eq1, eq2))

def replace(symbols, vals):
    for val, i in zip(vals, range(len(symbols))):
        if symbols[i] == val:
            symbols[i] = vals[val]
    return symbols, len(symbols) - len(vals)

symbols = make_symbols(cls1)
eq1 = make_eq(symbols, cls1)
eq2 = make_eq(symbols, cls2)
# sym_number = len(symbols)
# while sym_number != 1:
#     solution = solve_eq(eq1, eq2)
#     symbols, sym_number = replace(symbols, solution[0])
#     eq1 = make_eq(symbols, cls1)
#     eq2 = make_eq(symbols, cls2)
#     print(sym_number)

print(eq1)
print(eq2)
print(nsolve((eq2, 1), tuple(symbols), (3,1)))



(x0 - 2)**2/25 + (x1 - 2)**2/25
(x0 + 2)**2 + (x1 + 2)**2


ZeroDivisionError: matrix is numerically singular